In [2]:
!pip install google-genai pydantic

In [1]:
import os
os.environ['GEMINI_API_KEY'] = 'MY_API_KEY'

In [3]:
import os
from google import genai
from pydantic import BaseModel, Field
from enum import Enum

# Initialize the Google GenAI client (uses GEMINI_API_KEY from environment)
client = genai.Client()
MODEL_ID = 'gemini-2.5-flash'

# --- 1. Define the Swarm Network & Structured Outputs ---

class SwarmRole(str, Enum):
    MARKET = "Market Researcher"
    ENGINEER = "Engineering Agent"
    FINANCE = "Financial Modeling Agent"
    FINISH = "Consensus Reached (Finish)"

class DispatcherDecision(BaseModel):
    first_agent: SwarmRole = Field(description="The agent best suited to start the task.")
    reasoning: str = Field(description="Why this agent was chosen to start.")

class SwarmHandoff(BaseModel):
    contribution: str = Field(description="Your detailed analysis, proposal, or critique based on the current debate.")
    next_agent: SwarmRole = Field(description="The next agent to hand off to, or FINISH if a final design specification balancing all requirements has been achieved.")

# --- 2. Define the Agent Definitions ---

AGENT_PROMPTS = {
    SwarmRole.MARKET: "You represent consumer demands. Focus on what features users want and market trends.",
    SwarmRole.ENGINEER: "You represent technical feasibility. Focus on building the product, materials, and technical constraints.",
    SwarmRole.FINANCE: "You represent the budget. Focus on manufacturing costs, profit margins, and financial viability."
}

def swarm_node_agent(role: SwarmRole, conversation_history: str) -> SwarmHandoff:
    """A generic node function representing any agent in the swarm."""
    print(f"      -> [{role.value}] Analyzing and drafting contribution...")

    prompt = f"""You are the {role.value}.
    Your Directive: {AGENT_PROMPTS.get(role, "")}

    Current Swarm Debate & History:
    {conversation_history}

    Task:
    1. Read the history. Add your expert contribution, critique proposals from other agents, or build upon their ideas.
    2. Determine who needs to speak next (handoff).
    3. If the team has collectively converged on a final, balanced design specification, hand off to 'FINISH'.
    """

    response = client.models.generate_content(
        model=MODEL_ID,
        contents=prompt,
        config={
            'response_mime_type': 'application/json',
            'response_schema': SwarmHandoff,
            'temperature': 0.4, # Slightly higher temperature for creative debate
        },
    )
    return response.parsed

# --- 3. Define the Dispatcher Agent ---

def dispatcher_agent(user_request: str) -> SwarmRole:
    """Interprets the user request and determines which agent begins."""
    print("-> [Dispatcher] Interpreting request to find the best starting agent...")

    prompt = f"""Determine which specialized agent should begin this product design task.
    User Request: {user_request}"""

    response = client.models.generate_content(
        model=MODEL_ID,
        contents=prompt,
        config={
            'response_mime_type': 'application/json',
            'response_schema': DispatcherDecision,
        },
    )
    decision = response.parsed
    print(f"-> [Dispatcher] Routing to {decision.first_agent.value}. Reason: {decision.reasoning}\n")
    return decision.first_agent

# --- 4. The Swarm Environment (Execution Loop) ---

def swarm_workflow(user_request: str, max_iterations: int = 5) -> str:
    """Manages the swarm environment, shared memory, and explicit exit conditions."""
    print(f"Starting Swarm Workflow for: '{user_request}'\n")

    # Initialize the shared context/memory
    shared_history = f"User Request: {user_request}\n\n"

    # The dispatcher routes the query to an agent in the swarm to begin
    current_agent_role = dispatcher_agent(user_request)

    iteration = 1

    # Explicit Exit Condition: Max iterations to prevent unproductive loops
    while iteration <= max_iterations:
        print(f"--- Swarm Iteration {iteration} | Active Agent: {current_agent_role.value} ---")

        # Agent executes its task based on the shared history
        handoff_data = swarm_node_agent(current_agent_role, shared_history)

        print(f"   [Contribution]: {handoff_data.contribution[:100]}...\n")

        # Append the agent's contribution to the shared history for others to see
        shared_history += f"--- {current_agent_role.value} ---\n{handoff_data.contribution}\n\n"

        # Explicit Exit Condition: Consensus reached
        if handoff_data.next_agent == SwarmRole.FINISH:
            print("-> ✅ Swarm consensus reached! Returning final response.\n")
            return shared_history

        # Handoff: The agent determines who is better suited to handle the next step
        print(f"   [Handoff] -> Routing next to: {handoff_data.next_agent.value}\n")
        current_agent_role = handoff_data.next_agent
        iteration += 1

    print("-> ⚠️ Swarm did not converge. Max iterations reached. Returning current state.\n")
    return shared_history

# --- Execution ---
if __name__ == "__main__":
    # Ambiguous, highly complex problem
    task = "Design a smart, heated winter jacket that charges via solar panels."

    # Trigger the swarm
    final_debate_log = swarm_workflow(task)

    print("=== Final Swarm Consensus Log ===")
    print(final_debate_log)

Starting Swarm Workflow for: 'Design a smart, heated winter jacket that charges via solar panels.'

-> [Dispatcher] Interpreting request to find the best starting agent...
-> [Dispatcher] Routing to Market Researcher. Reason: To design a new product like a smart, heated winter jacket with solar charging, it is crucial to first understand the market demand, target audience, competitive landscape, and potential feature set. The Market Researcher can identify existing gaps, validate the product concept, and determine the most desired features before any engineering or financial modeling begins.

--- Swarm Iteration 1 | Active Agent: Market Researcher ---
      -> [Market Researcher] Analyzing and drafting contribution...
   [Contribution]: The concept of a smart, heated winter jacket with solar charging is highly appealing to consumers, e...

   [Handoff] -> Routing next to: Engineering Agent

--- Swarm Iteration 2 | Active Agent: Engineering Agent ---
      -> [Engineering Agent] Analyzi